2. Consider the contraction hierarchies presented during the course. Assume
to deal with graphs that can be fully represented in the memory of your
computer. Implement:

* (a) an algorithm to add the shortcuts to a graph;
* (B) a bidirectional version of Dijkstra algorithm that can operate on the
graphs decorated by the algorithm at Point 2a.

We import some necessary libraries which will aid us in better implementing the algorithms.

In [1]:
from binheap import binheap
from math import inf
from collections import defaultdict
from copy import deepcopy

This algorithm demands a richer structure in terms of classes since one must be able to remove nodes, add shortcuts, and reverse a graph in order to go "backwards" in the bidirectional version of Dijkstra. For this reason, I re-implemented the vertex-graph classes, adding also an edge class. A vertex has an id, the key, the heap index, the importance, and neighboring vertices. Furthemore, in order to add shortcuts one must keep track of the vertices for which a specific vertex is a tail of the edge and those for which it is a head of the edge, this is done by **src** and **dest** attributes respectively.

It is important to note that not all of the functionalities of the classes are strictly necessary but I added them for completeness. 


In [2]:
class Vertex:
    def __init__(self, name, imp = 0, key = inf):
        self.id = name
        self.key = key
        self.heap_idx = None
        self.imp = imp
        self.visited = False
        self.neighbors = {}
        self.src = {}        #nodes for which vertex is tail (thus these are the heads of the vertex)
        self.dest = {}       #nodes for which vertex is head (thus these are the tails of the vertex)
    
    def __repr__(self):
        return str(self.id)

    ############################## UTILITY FUNCTIONS ###################################


    def add_neighbor(self, node, weight = 0): #add an edge between two vertices
        '''Input: destination Vertex and weight of corresponding edge. 
        Stores an edge'''
        self.neighbors[node.id]=Edge(self, node, weight)
        self.src[node.id] = node
        node.dest[self.id] = self
        
    def delete_neighbor(self, node):
        '''Delete edge between a node and its neighbor'''
        self.neighbors.pop(node.id, None)
        self.src.pop(node.id, None)
        self.dest.pop(node.id, None)

    def get_tails(self):
        '''returns tails of a vertex'''
        return self.dest.values()

    def get_heads(self):
        '''returns head of a vertex'''
        return self.src.values()

    def get_neighbor(self, node): #returns an edge
        '''Input: destination Vertex.
        Output: Edge() between both vertices.'''
        if self.neighbors.get(node.id) is None:
            return None
        else: 
            return self.neighbors[node.id]

    def get_neighbors(self):
        '''returns all edges of a vertex'''
        return self.neighbors.values() #returns all edges

    def get_neighbor_keys(self):
        '''returns keys of neighbors'''
        return self.neighbors.keys()
    
    def reverse_edge(self, node):
        '''reverse an edge between self and a node by creating a new reversed edge'''
        node.neighbors[self.id] = self.neighbors[node.id].create_reversed() 
        self.neighbors.pop(node.id) #eliminiate original edge

        self.src.pop(node.id)
        node.dest.pop(self.id)

        self.dest[node.id] = node
        node.src[self.id] = self


    ############################## OPERATOR OVERLOADING ##################################

    def impcomp(self, other):
        """Compares importances between two nodes."""
        return self.imp <= other.imp 


    ############################# GETTERS AND SETTERS ###################################


    def get_id(self):
        return self.id

    def set_key(self, key):
        self.key = key

    def get_key(self):
        return self.key

    def set_imp(self, imp):
        self.imp = imp

    def get_imp(self):
        return self.imp

    def set_visited(self):
        self.visited = True

    def get_visited(self):
        self.visited


In [3]:
class Edge:
    def __init__(self, src:Vertex, dest:Vertex, weight=0):
        self.weight = weight
        self.src = src
        self.dest = dest
        
    def __repr__(self):
        return 'EDGE ' + str(self.src.get_id()) + '->' + str(self.dest.get_id()) + ' Weight ' + str(self.weight)
    
    ################################  UTILITY FUNCTIONS   ##################################

    def create_reversed(self):                              #creates reversed copy and returns it
        return Edge(self.dest,self.src, self.weight)

    ################################# GETTERS AND SETTERS ###################################
    def get_src(self):
        return self.src
    def get_dest(self):
        return self.dest
    def set_weight(self, weight):
        self.weight = weight
    def get_weight(self):
        return self.weight
    def get_src_imp(self):
        return self.src.get_imp()
    def get_src_key(self):
        return self.src.get_key()
    def set_src_key(self, key):
        self.src.set_key(key)
    def get_dest_imp(self):
        return self.dest.get_imp()
    def get_dest_key(self):
        return self.dest.get_key()
    def set_dest_key(self, key):
        self.dest.set_key(key)


Below we implement the graph class. It is similar to the one in precedence except it adds neighbors to a vertex directly as opposed to adding the tuple (destination, weight) as before. 

Furthermore, point (a) is implemented in the methods **remove_node()** and **add_shortcut()**. The first method adds shortcuts and actually removes the node from the graph structure while the second method simply adds the necessary shortcuts to the graph without actually removing the node. The first method can be used to build the overlay graphs we studied in class (in this case the method modifies the graph itself but it can be easily modified to return a graph instead) while the second method, if iterated over all the edges, can be used to build the contraction hierarchy with all edges present (including all shortcuts).

In [4]:
class Graph: #collection of edges and vertexes
    def __init__(self):
        self.vertex = {}

    def __repr__(self):
        st = ''
        for v in self.vertex.values():
            hd = str(v)
            level = '\t'.join(f'{n}' for n in v.get_neighbors())
            st += hd + ': ' + level + f'\n'
        return st

    def add_vertex(self, node, imp=0): #insert vertex into graph with given importance
        vert = Vertex(node, imp) #creates vertex and appends it to the dictionary
        self.vertex[vert.id] = vert

    def get_vertex(self, node):
        '''get vertex from graph'''
        if isinstance(node, Vertex):
            return self.vertex[node.id]
        else:
            return self.vertex[node]

    def get_vertices(self):
        '''get all vertices'''
        return self.vertex.values()


    def add_edge(self, src, dest, weight=0, imp1 = 0, imp2 = 0):
        '''add edge between src and dest with specified weight and importances if the vertices have not been created yet'''
        if src not in self.vertex.keys():
            self.add_vertex(src, imp1)
        if dest not in self.vertex.keys():
            self.add_vertex(dest, imp2)
        self.vertex[src].add_neighbor(self.vertex[dest], weight)

    def get_edges(self, node):
        '''get edges of a vertex node'''
        if isinstance(node, Vertex):
            return self.vertex[node.id].get_neighbors()
        else:
            return self.vertex[node].get_neighbors()

    def get_alledges(self):
        '''get all edges of the graph'''
        edges=[]
        for v in self.get_vertices():
            edges.extend(list(self.vertex[v.id].get_neighbors()))
        return edges

    def get_edge(self, node1:Vertex, node2:Vertex):
        '''get a specific edge between two vertices'''
        return self.vertex[node1.id].get_neighbor(node2)

    def reverse_edge(self, node1: Vertex, node2: Vertex):
        '''reverse a specific edge between two vertices'''
        self.vertex[node1.id].reverse_edge(node2)
    
    def remove_node(self, node: Vertex):
        '''remove node and add shortcuts if needed'''
        tails = list(self.get_vertex(node).get_tails())
        heads = list(self.get_vertex(node).get_heads())
        if tails and heads:
            for src in tails:
                for dest in heads:
                    if src is not dest:
                        n_W = self.get_edge(src, node).get_weight() + self.get_edge(node, dest).get_weight()
                        if self.get_edge(src, dest) is None:
                            self.add_edge(src.get_id(), dest.get_id(), n_W)
                            
                        else: 
                            o_W = self.get_edge(src, dest).get_weight()
                            W = min(n_W,o_W)
                            self.get_edge(src, dest).set_weight(W)
                        self.get_vertex(dest).delete_neighbor(node)

                self.get_vertex(src).delete_neighbor(node)
        self.vertex.pop(node.id)
         
    def add_shortcut(self, node: Vertex): 
        '''add shortcut to the graph as if you removed the node without actually removing it'''
        tails = list(self.get_vertex(node).get_tails())
        heads = list(self.get_vertex(node).get_heads())
        if tails and heads:
            for src in tails:
                for dest in heads:
                    if src is not dest:
                        n_W = self.get_edge(src, node).get_weight() + self.get_edge(node, dest).get_weight()
                        if self.get_edge(src, dest) is None:
                            self.add_edge(src.get_id(), dest.get_id(), n_W)   
                        else: 
                            o_W = self.get_edge(src, dest).get_weight()
                            W = min(n_W,o_W)
                            self.get_edge(src, dest).set_weight(W)


To implement bidirectional Dijsktra we also need to be able to reverse a graph so that we can traverse edges backwards from the destination. This is done below.

In [5]:
def reverse_graph(d):
    '''create a reversed copy of the graph'''
    g = Graph()
    edges = deepcopy(d.get_alledges())
    for edge in edges:
        src = edge.get_src()
        dest = edge.get_dest()
        g.add_edge(dest.id, src.id, edge.get_weight(), dest.imp, src.imp)
    return g

Finally, we implement a function which takes a graph and adds all shortcuts. In this way we construct the contraction hierarchy which is necessary for the bidirectional algorithm.

Furthermore, we also implement a **PREP()** function which acts a preprocessing step for the graph. In other words, given a graph, we build the contraction hierarchy and then build the reverse graph. In fact, the contraction hierarchy and the reversed graph will be some of the inputs for the bidirectional Dijkstra's algorithm. 

In [6]:
def CH(d):   #adds all shortcuts
    '''adds all shortcuts to a graph'''
    g = deepcopy(d)
    edges = g.get_alledges()
    for e in edges:
        if e.get_dest().impcomp(e.get_src()):
            g.add_shortcut(g.get_vertex(e.get_dest().id))
    return g
    

In [7]:
def PREP(g):
    '''takes a graph, adds all shortcuts and creates reversed copy
    returns: contraction hierarchy and reversed graph'''
    d = CH(g)
    drev = reverse_graph(d)
    return d, drev

Below we implement the bidirectional version of Dijkstras algorithm. The inputs are the contraction hierarchy, the reversed graph, the source vertex and the destination vertex.

In [8]:
def bidir_dijkstra(g, grev, src: Vertex, dest: Vertex):
    '''bidirectional dijkstra'''
    dist = {}
    distrev = {}

    g.get_vertex(src).set_key(0)     #set key to zero for src
    grev.get_vertex(dest).set_key(0) #set key to zero for dest

    BH = binheap(list(g.get_vertices())) #create heaps
    BHrev = binheap(list(grev.get_vertices()))
   
    for num, node in enumerate(BH._A): #identify heap index with the node attribute to keep track.
        node.heap_idx = num

    for num, node in enumerate(BHrev._A):
        node.heap_idx = num


    while not set(dist.keys()).intersection(set(distrev.keys())): #while intersection is empty do:
        
        nf = BH.remove_minimum()
        nb = BHrev.remove_minimum()
        #forward step
        if nf.get_key() is not inf:
            nf.set_visited()
 
            dist[nf.id]=nf.get_key()

            for edge in list(g.get_edges(nf.id)):
                if edge.get_dest().get_visited():
                    continue
                # only update key if the dest node is more important 
                # and if they key update is less than what it was.
                # The last part is exactly as the previous case
                if nf.impcomp(edge.get_dest()) and dist[nf.id]+edge.get_weight() < edge.get_dest_key() :
                    BH.decrease_key(edge.get_dest().heap_idx, dist[nf.id]+edge.get_weight()) 
        #backward step
        if nb.get_key() is not inf:
            nb.set_visited() 
            distrev[nb.id]=nb.get_key()

            for edge in list(grev.get_edges(nb.id)):
                if edge.get_dest().get_visited():
                    continue
                if nb.impcomp(edge.get_dest()) and distrev[nb.id]+edge.get_weight() < edge.get_dest_key():
                    BHrev.decrease_key(edge.get_dest().heap_idx, distrev[nb.id]+edge.get_weight())
    #find the intersection
    node = set(dist.keys()).intersection(set(distrev.keys()))
    for i in node:
        w = dist[i] + distrev[i]
    return w, dist, distrev #return the shortest distance and the dictionary of finalized nodes


We test the implementation on a simple graph according to the following figure.

![Title](./es44.png)

The shortest path from **a** to **h** is clearly 12. However, since we use the contraction hierarchy and only move to nodes of higher importance, during the dijkstra algorithm we can "skip" looking over **b** and move directly to **c** and skip looking at **d** and move directly to **e**. Similarly, in the backward pass we save time by not looking at some nodes and get to **e** faster, which is where the two paths meet. At this point we simply sum up the distances gathered so far and return the result. 

Below we implement the graph and inspect its correcteness:

In [9]:
g = Graph()
g.add_vertex('a', 2)
g.add_vertex('b', 1)
g.add_vertex('c', 4)
g.add_vertex('d', 3)
g.add_vertex('e', 5)
g.add_vertex('f', 2.5)
g.add_vertex('g', 1.2)
g.add_vertex('h', 2.7)

g.add_edge('a', 'b', 1)
g.add_edge('b', 'c', 2)
g.add_edge('c', 'd', 1)
g.add_edge('d', 'e', 3)
g.add_edge('e', 'f', 2)
g.add_edge('f', 'g', 1)
g.add_edge('g', 'h', 2)

In [10]:
g

a: EDGE a->b Weight 1
b: EDGE b->c Weight 2
c: EDGE c->d Weight 1
d: EDGE d->e Weight 3
e: EDGE e->f Weight 2
f: EDGE f->g Weight 1
g: EDGE g->h Weight 2
h: 

Now we apply Dijkstra's bidirectional algorithm:

In [11]:
d, drev = PREP(g)
pathdist, dist, distrev = bidir_dijkstra(d, drev, g.get_vertex('a'), g.get_vertex('h'))

In [12]:
pathdist

12

As we can see, the result is correct just as we expected. Furthermore, by analyzing the dist and distrev dictionaries we can get a sense of the nodes that the algorithm looked at. 

In [13]:
dist

{'a': 0, 'c': 3, 'e': 7}

In [14]:
distrev

{'h': 0, 'e': 5}

As mentioned above, in the forward pass, the algorithm went from **a** directly to **c** and then directly to **e**. In the backward pass instead, the algorithm went from the destination directly to **e**. This tells us that the algorithm is indeed looking at a reduced number of nodes and thus is able to perform faster. Unfortunately however, in my implementation this comes at a cost of memory. In fact, as we can see, we have to create a graph which is much denser (the contraction hierarchy) as well as the reverse graph. However, since we assumed from the start that the entire graph fits in memory, this was not the main concern. 

Now we test on a more complicated graph such as the one shown below (all weights are set to 1). If we want the shortest path distance from **a** to **f**, it is clear that it will have a value of 4 and will have the following path: a -> d -> c -> h -> f. 

![Title](./es33.png)

We create the graph

In [15]:
g = Graph()
g.add_vertex('a', 1)
g.add_vertex('b', 3)
g.add_vertex('c', 4)
g.add_vertex('d', 2)
g.add_vertex('e', 4)
g.add_vertex('f', 3.5)
g.add_vertex('g', 2)
g.add_vertex('h', 1.5)

g.add_edge('a', 'b', 1)
g.add_edge('a', 'd', 1)
g.add_edge('b', 'd', 1)
g.add_edge('c', 'b', 1)
g.add_edge('c', 'h', 1)
g.add_edge('c', 'e', 1)
g.add_edge('d', 'c', 1)
g.add_edge('e', 'd', 1)
g.add_edge('e', 'g', 1)
g.add_edge('f', 'a', 1)
g.add_edge('g', 'f', 1)
g.add_edge('g', 'h', 1)
g.add_edge('h', 'f', 1)

We inspect its correctness:

In [16]:
g

a: EDGE a->b Weight 1	EDGE a->d Weight 1
b: EDGE b->d Weight 1
c: EDGE c->b Weight 1	EDGE c->h Weight 1	EDGE c->e Weight 1
d: EDGE d->c Weight 1
e: EDGE e->d Weight 1	EDGE e->g Weight 1
f: EDGE f->a Weight 1
g: EDGE g->f Weight 1	EDGE g->h Weight 1
h: EDGE h->f Weight 1

We can see that the graph is accurately represented.

Now we apply Dijkstra's bidirectional algorithm. 

In [17]:
d, drev = PREP(g)
pathdist, dist, distrev = bidir_dijkstra(d, drev, d.get_vertex('a'), d.get_vertex('f'))

In [18]:
pathdist

4

As expected, the shortest distance is indeed 4. However in this case, since we start from **a** the algorithm can go to any other node since they are all more important than **a**. This means that we can have some backtracking which is indeed the case as we can observe in the dist and distrev dictionaries:

In [19]:
dist

{'a': 0, 'b': 1, 'd': 1, 'c': 2}

We can see that in the forward pass the algorithm started from **a** then looked at **b**, then **d** and finally **c** by taking the shortcut which occurs if you eliminate **d** (which is why **c** has a distance of 2 even though all weights are set to 1).

In [20]:
distrev

{'f': 0, 'c': 2, 'e': 2}

On the other hand, in the backward pass, the algorithm started form the destination **f**, and went directly to **c** and then **e** which is useless for the path since the forward and backward pass meet at vertex **c**.
This confirms that our algorithm is working properly as desired. In fact, the algorithm stops when the backward and forward pass have a common key in the dictionary keys. As soon as this happens, we are sure that the path is indeed the shortest one, as explained during class. 